# Imports

In [1]:
import torch 
print('torch.cuda.is_available:', torch.cuda.is_available())
print('torch.cuda.device_count:', torch.cuda.device_count())

torch.cuda.is_available: True
torch.cuda.device_count: 2


In [2]:
import numpy as np
import torch
torch.set_float32_matmul_precision('high')
from train import *

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

# Experiment settings
Settings are based on the training procedure that produced the final generator used in evaluating the stylized facts and training the RL agent in portfolio management.  
However, using different machines would result in cumulative differences in computation results likely due to varied machine precision.  
Original experiments were performed on machine with following specs:
1. OS: Ubuntu 20.04.1
2. CPU: Intel(R) Xeon(R) Gold 6248 CPU @ 2.50GHz
3. GPU: Tesla V100-SXM2-32GB

### Dataset related

In [3]:
# samples
batch_size = 38 # number of samples in each batch
sample_len = 300 # length of each sample
sample_model = 'Realdt' # GBM, Heston, OU, RealData, Realdt, spx_rates
lead_lag = True # whether to use lead lag transformation
lags = [1] # number of lags to use for lead lag transformation: int or list[int]
seed = 42

# real data parameters
stride = 50 # for real data
start_date = '1995-01-01' # start date for real data
end_date = '2018-09-18' # end date for real data

### Generator and kernel related

In [4]:
# signature kernel
static_kernel_type = 'rq' # type of static kernel to use - rbf, rbfmix, rq, rqmix, rqlinear for
n_levels = 10 # number of levels in the truncated signature kernel

# generator
seq_dim = 1 # dimension of sequence vector
activation = 'Tanh' # pytorch names e.g. Tanh, ReLU. NOTE: does NOT change transformer layers'
hidden_size = 64
n_lstm_layers = 1 # number of LSTM layers
conditional = True # feed in history for LSTM generators
hist_len = 50

### Noise related

In [5]:
noise_dim = 4 # dimension of noise vector
ma = True # whether to use MA noise generator fitted to log returns gaussianized by Lambert W transformation
ma_p = 20

### Training Related

In [6]:
epochs = 10000 # number of batches
start_lr = 0.001 # starting learning rate
patience = 100 # number of epochs to wait before reducing lr
lr_factor = 0.5 # factor to multiply lr by for scheduler
early_stopping = patience*3 # number of epochs to wait before no improvement
kernel_sigma = 0.1 # starting kernel_sigma
num_losses = 20

### Save to tensorboard log

In [7]:
# save all parameters to a dictionary
rng = np.random.default_rng(seed)
torch.manual_seed(seed)

data_params, model_params, train_params = get_params_dicts(vars().copy())

# save parameters to tensorboard
writer = start_writer(data_params, model_params, train_params)

### Data, kernel, generator

In [8]:
dataloader = get_dataloader(**{**data_params, **model_params})
kernel = get_signature_kernel(**{**model_params, **train_params})
generator = get_generator(**{**model_params, **data_params})
generator.to(device)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5855.988337133396
            Iterations: 36
            Function evaluations: 836
            Gradient evaluations: 36
                        Zero Mean - ARCH Model Results                        
Dep. Variable:           gaussianized   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -5855.99
Distribution:                  Normal   AIC:                           11754.0
Method:            Maximum Likelihood   BIC:                           11897.9
                                        No. Observations:                 6999
Date:                Sun, Sep 01 2024   Df Residuals:                     6999
Time:                        18:51:36   Df Model:                            0
                               Volatility Model                              
 

GenLSTM(
  (rnn): LSTM(6, 64, batch_first=True)
  (mean_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (var_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (output_net): Linear(in_features=64, out_features=1, bias=True)
)

# Train MMD-GAN

In [9]:
train(generator, kernel, dataloader, rng, writer, device, **{**train_params, **model_params, **data_params})

/u/hous/Generative-Model-Signature-MMD/.venv/lib64/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
100%|██████████| 3/3 [00:07<00:00,  2.36s/it]


Epoch 0, loss: 709563.8958333334, avg_last_20_loss: 709563.8958333334
Saving model at epoch 0


100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


Epoch 1, loss: 473359.2083333333, avg_last_20_loss: 591461.5520833334
Saving model at epoch 1


100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


...

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


Epoch 93, loss: 18072.0244140625, avg_last_20_loss: 22024.016894531247
Saving model at epoch 93


100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


Epoch 94, loss: 16158.642252604166, avg_last_20_loss: 21686.788899739586
Saving model at epoch 94


 67%|██████▋   | 2/3 [00:02<00:01,  1.49s/it]